In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [2]:
anon = pd.read_csv('../Data_Anom/anonym212_407', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [3]:
print(anon)

             id                date  latitude  longitude  week
0         14903 2015-04-25 23:06:00  4.866097  45.755977    17
1         76458 2015-04-16 07:35:00  2.778187  48.873183    16
2         63452 2015-03-28 08:30:00  4.890675  45.769787    13
3         40778 2015-04-02 11:24:00  4.890618  45.769671    14
4         56820 2015-03-26 14:47:00  4.884803  45.779076    13
...         ...                 ...       ...        ...   ...
34551844  26824 2015-04-28 05:12:00  4.868006  45.755944    18
34551845  89752 2015-03-07 13:13:00  5.279579  45.585559    10
34551846  78658 2015-03-05 15:50:00  4.892105  45.765921    10
34551847  95512 2015-04-23 21:10:00  5.112896  43.985684    17
34551848  10926 2015-03-17 15:09:00  4.875839  45.770824    12

[30784945 rows x 5 columns]


In [4]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [5]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['10131' '10145' '10332' '10926' '11076' '11365' '12124' '12659' '12674'
 '12701' '12887' '13010' '13502' '14115' '1415' '14183' '14571' '14672'
 '14903' '15008' '15016' '15028' '15323' '15383' '15487' '16087' '1613'
 '16865' '16901' '17048' '17695' '17820' '17828' '18518' '18776' '19101'
 '19125' '1916' '19252' '19384' '19421' '19447' '19871' '20118' '20232'
 '20250' '20637' '2078' '20799' '21661' '21743' '21883' '21975' '21988'
 '22053' '22347' '22689' '22797' '22891' '23073' '23672' '24620' '24793'
 '25200' '25218' '25246' '26152' '26359' '26386' '26739' '26770' '26824'
 '26844' '26925' '27140' '27240' '27334' '27437' '27539' '28159' '28290'
 '28315' '28316' '28484' '28859' '28868' '29021' '29044' '29050

In [6]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [43]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2, 1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])),4)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600),1)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)) ,1/2)
            
            list_distance.append(distance*delta_rati*distance) #*delta_ratio*delta_time*distance*delta_hour
            
        index = list_distance.index(min(list_distance))
        l = id_week_anon[group_week.id[index]]# list of week have id anon_mean.id[index] 
        while ((w not in l)):
            list_distance[index] = 1000000000
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
print(result)

391
{'1': {'2015-10': ['45798'], '2015-11': ['33720'], '2015-12': ['5584'], '2015-13': ['48490'], '2015-14': ['3712'], '2015-15': ['94286'], '2015-16': ['70624'], '2015-17': ['80894'], '2015-18': ['42257'], '2015-19': ['8002'], '2015-20': ['88732']}, '2': {'2015-10': ['17828'], '2015-11': ['4554'], '2015-12': ['21988'], '2015-13': ['77136'], '2015-14': ['65264'], '2015-15': ['70404'], '2015-16': ['94112'], '2015-17': ['32932'], '2015-18': ['59809'], '2015-19': ['20232'], '2015-20': ['75762']}, '107': {'2015-10': ['9429'], '2015-11': ['65900'], '2015-12': ['71275']}, '98': {'2015-10': ['16865'], '2015-11': ['26925']}, '21': {'2015-10': ['28316'], '2015-11': ['77479']}, '5': {'2015-10': ['3045'], '2015-11': ['14183']}, '9': {'2015-10': ['97117'], '2015-11': ['11076'], '2015-12': ['98621'], '2015-13': ['80379'], '2015-14': ['40949'], '2015-15': ['95781'], '2015-16': ['33558'], '2015-19': ['8354']}, '6': {'2015-10': ['91964'], '2015-11': ['18518'], '2015-12': ['89351'], '2015-13': ['82693'

In [41]:
jsonData = json.dumps(result, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "45798"
        ],
        "2015-11": [
            "35034"
        ],
        "2015-12": [
            "5584"
        ],
        "2015-13": [
            "48490"
        ],
        "2015-14": [
            "3712"
        ],
        "2015-15": [
            "94286"
        ],
        "2015-16": [
            "70624"
        ],
        "2015-17": [
            "80894"
        ],
        "2015-18": [
            "42257"
        ],
        "2015-19": [
            "8002"
        ],
        "2015-20": [
            "88732"
        ]
    },
    "2": {
        "2015-10": [
            "17828"
        ],
        "2015-11": [
            "4554"
        ],
        "2015-12": [
            "21988"
        ],
        "2015-13": [
            "77136"
        ],
        "2015-14": [
            "65264"
        ],
        "2015-15": [
            "70404"
        ],
        "2015-16": [
            "94112"
        ],
        "2015-17": [
            "3293